In [3]:
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [4]:
drug_train = pd.read_csv("Data/drugLibTrain_raw.tsv",delimiter='\t',encoding='utf-8')
drug_test = pd.read_csv("Data/drugLibTest_raw.tsv",delimiter='\t',encoding='utf-8')

In [5]:
drug=pd.concat([drug_train,drug_test], axis=0, join='outer', ignore_index=False, keys=None,
          levels=None, names=None, verify_integrity=False, copy=True)

In [6]:
drug.shape

(4143, 9)

In [7]:
drug.columns

Index(['Unnamed: 0', 'urlDrugName', 'rating', 'effectiveness', 'sideEffects',
       'condition', 'benefitsReview', 'sideEffectsReview', 'commentsReview'],
      dtype='object')

In [8]:
drug['commentsReview'][3]

3    I was given Prilosec prescription at a dose of...
3    This drug may not be for everyone but its wond...
Name: commentsReview, dtype: object

In [9]:
import pandas as pd
import os
folder = 'aclImdb'
labels = {'pos': 1, 'neg': 0}
df = pd.DataFrame()
for f in ('test', 'train'):    
    for l in ('pos', 'neg'):
        path = os.path.join(drug, f, l)
        for file in os.listdir (path) :
            with open(os.path.join(path, file),'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]],ignore_index=True)
df.columns = ['review', 'sentiment']

TypeError: expected str, bytes or os.PathLike object, not DataFrame

In [10]:
reviews=drug['commentsReview'].str.split(' ', expand=True)
tokens = word_tokenize(reviews)


TypeError: expected string or bytes-like object

In [11]:
reviews.head()

,0,1,2,3,4,5,6,7,8,9,...,752,753,754,755,756,757,758,759,760,761
0,monitor,blood,pressure,",",weight,and,asses,for,resolution,of,...,None,None,None,None,None,None,None,None,None,None
1,I,Hate,This,Birth,"Control,",I,Would,Not,Suggest,This,...,None,None,None,None,None,None,None,None,None,None
2,I,took,2,pills,at,the,onset,of,my,menstrual,...,None,None,None,None,None,None,None,None,None,None
3,I,was,given,Prilosec,prescription,at,a,dose,of,45mg,...,None,None,None,None,None,None,None,None,None,None
4,See,above,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [17]:
drug.head()

,Unnamed: 0,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,2202,enalapril,4,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ..."
1,3117,ortho-tri-cyclen,1,Highly Effective,Severe Side Effects,birth prevention,Although this type of birth control has more c...,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...","I Hate This Birth Control, I Would Not Suggest..."
2,1146,ponstel,10,Highly Effective,No Side Effects,menstrual cramps,I was used to having cramps so badly that they...,Heavier bleeding and clotting than normal.,I took 2 pills at the onset of my menstrual cr...
3,3947,prilosec,3,Marginally Effective,Mild Side Effects,acid reflux,The acid reflux went away for a few months aft...,"Constipation, dry mouth and some mild dizzines...",I was given Prilosec prescription at a dose of...
4,1951,lyrica,2,Marginally Effective,Severe Side Effects,fibromyalgia,I think that the Lyrica was starting to help w...,I felt extremely drugged and dopey. Could not...,See above


In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [39]:
t = drug[['commentsReview','benefitsReview']].apply(lambda x: ' '.join(str(s) for s in x), axis=1)

In [46]:
c=CountVectorizer(max_features=500,binary=True)
j = c.fit_transform(t)
X = j.toarray()

In [ ]:
# np .random.seed(0)

# data = np.random.rand(200, 4)
# data = np.round(data, decimals=0)
# target = np.random.randint(1, 4, size=(200, 1))
# data=np.concatenate((data,target),axis=1)
# X_train = data[:150, :-1]
# y_train = data[:150, -1]
# X_test = data[150:, :-1]
# y_test = data[150:, -1]

In [ ]:
class SoftmaxRegresion:
    def __init__(self, 
                 lr=0.0001, 
                 epoch=1,
                 tolerence=10e-8,
                 epsilon=10e-8,
                 minibatchsize=30):
        self.lr=lr
        self.epoch=epoch
        self.epsilon=epsilon
        self.tolerence=tolerence
        self.minibatchsize=minibatchsize
        
    
    
    
    def sigmoid(self,X,theta):
        #print('shape of X', X.shape)
        #print('shape pg theta', theta.shape)
        z = X@theta
        return 1/(1 + np.exp(-z))
    
    # compute the grandient  
    def get_gradientB(self,X,y,theta):
        h   = self.sigmoid(X,theta)
        return np.dot(X.T, (h - y))
    
    def get_gradientS(self,X,y,theta):
        return (1/len(X))*np.sum(( self.sigmoid(X,theta) - y)*X)
    
    #compute the loss
    def loss(self,X,y,theta):
        return   (-y*np.log(self.sigmoid(X,theta))  - (1-y)*np.log(1- self.sigmoid(X,theta))).mean()
            
        
        
    #mini batch gradient descent        
    def fit(self,X,y):
        
        #make a copy of our data
        X_copy=X.copy()
        y_copy=y.copy()
        y_enc = (np.arange(np.max(y_copy) + 1) == y[:, None]).astype(float)

        
         #add the intercept column
        intercept=np.ones((X_copy.shape[0],1))
        X_copy=np.concatenate((intercept,X_copy),axis=1)
        #print('shape of the whole x',X_copy.shape)
        
        #initialise the weight
        self.theta=np.zeros((X_copy.shape[1],1))
        #print('shape of theta',theta.shape)
        diff=1
        current_iter=1
        minibatch = int(len(X_copy)/self.minibatchsize)
        #while diff >= self.tolerence current_iter<self.max_iter :    
        for i in range(self.epoch):
            random_vector=np.random.permutation(X_copy.shape[0])
            #random_vectory=np.random.permutation(y_copy.shape[0])
            X_cop=X_copy[random_vector]
            y_cop=y_copy[random_vector]
            #print('X_ shape',X_cop.shape)
            #print('y_ shape',y_cop.shape)
            #prev_theta=self.theta.copy()
            for j in range(minibatch):
                X_=X_cop[j*self.minibatchsize:(j+1)*self.minibatchsize]
                y_=y_cop[j*self.minibatchsize:(j+1)*self.minibatchsize]
#                     print('X_ shape',X_.shape)
#                     print('y_ shape',y_.shape)
                #compute the gradient
                grad = self.get_gradientS(X_, y_, self.theta)

                self.theta = self.theta + self.lr*grad.reshape(-1,1)
                #print('shape of theta',self.theta.shape)
            #cur_theta=self.theta.copy()
            #diff=np.linalg.norm(prev_theta - cur_theta)
            #current_iter+=1
            #print('my update',self.theta)
            #print the value of the loss
            print('the loss function is ',self.loss(X_[j],y_[j],self.theta))

          
    
    
    
    def softmax(self,X, theta, bias):
        
        z=(X.dot(theta) + bias)
        print(z.shape)
        
        return (np.exp(z.T) / np.sum(np.exp(z), axis=1)).T
    
    #mini batch gradient descent        
    def fit_MBGD(self,X,y):
        
        #make a copy of our data
        X_copy=X.copy()
        y_copy=y.copy()
        
    
        self.theta=np.zeros((X_copy.shape[1],1))
        intercept=np.ones((1, self.theta.shape[1]))
        
        
         #add the intercept column
        
        
        #print('shape of the whole x',X_copy.shape)
        
        #initialise the weight
        
        #print('shape of theta',theta.shape)
        diff=1
        current_iter=1
        minibatch = int(len(X_copy)/self.minibatchsize)
        #while diff >= self.tolerence current_iter<self.max_iter :    
        for i in range(self.epoch):
            random_vector=np.random.permutation(X_copy.shape[0])
            #random_vectory=np.random.permutation(y_copy.shape[0])
            X_cop=X_copy[random_vector]
            y_cop=y_copy[random_vector]
            #print('X_ shape',X_cop.shape)
            #print('y_ shape',y_cop.shape)
            #prev_theta=self.theta.copy()
            for j in range(minibatch):
                X_=X_cop[j*self.minibatchsize:(j+1)*self.minibatchsize]
                y_=y_cop[j*self.minibatchsize:(j+1)*self.minibatchsize]
#                     print('X_ shape',X_.shape)
#                     print('y_ shape',y_.shape)
                #compute the gradient
                grad = self.softmax(X_, self.theta,intercept)
                #print(grad.shape)
                self.theta = self.theta + self.lr*grad.reshape(-1,1)
                #print('shape of theta',self.theta.shape)
            #cur_theta=self.theta.copy()
            #diff=np.linalg.norm(prev_theta - cur_theta)
            #current_iter+=1
            #print('my update',self.theta)
            #print the value of the loss
            print('the loss function is ',self.loss(X_[j],y_[j],self.theta))

        

    def predict(self,X):
        X_copy=X.copy()
        
        #print('shape of theta',self.theta.shape)
        intercept=np.ones((X_copy.shape[0],1))
        #print('shape of x copy' ,X_copy.shape)
        #print('shape of intercept' ,intercept.shape)
        X_copy=np.concatenate((intercept, X_copy), axis=1)
        #print('shape',X_copy.shape)
        
        return self.sigmoid(X_copy,self.theta)
 